In [0]:
 try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
import functools
import sys
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow_core.python.keras import regularizers
from tensorflow.keras import layers
import pandas as pd

In [0]:
train_file = "heart_train.csv"
test_file = "heart_test.csv"

In [260]:
!head {train_file}

row.names,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
56,152,5.99,7.99,32.48,Absent,45,26.57,100.32,48,0
57,118,0,2.99,16.17,Absent,49,23.83,3.22,28,0
58,126,5.1,2.96,26.5,Absent,55,25.52,12.34,38,1
59,103,0.03,4.21,18.96,Absent,48,22.94,2.62,18,0
60,121,0.8,5.29,18.95,Present,47,22.51,0,61,0
61,142,0.28,1.8,21.03,Absent,57,23.65,2.93,33,0
62,138,1.15,5.09,27.87,Present,61,25.65,2.34,44,0
63,152,10.1,4.71,24.65,Present,65,26.21,24.53,57,0
64,140,0.45,4.3,24.33,Absent,41,27.23,10.08,38,0


In [261]:
!head {test_file}

row.names,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
1,150,13.8,5.1,29.45,Present,52,27.92,77.76,55,1
2,176,6,3.98,17.2,Present,52,21.07,4.11,61,1
3,142,2.2,3.29,22.7,Absent,44,23.66,5.66,42,1
4,132,0,3.3,21.61,Absent,42,24.92,32.61,33,0
5,142,1.32,7.63,29.98,Present,57,31.16,72.93,33,0
6,146,1.16,2.28,34.53,Absent,50,28.71,45,49,0
7,132,7.2,3.65,17.16,Present,56,23.25,0,34,0
8,120,0,3.57,23.22,Absent,58,27.2,0,32,0
9,118,0,3.89,15.96,Absent,65,20.18,0,16,0


In [0]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [0]:
class PackNumericFeatures(object):
  def __init__(self,names):
    self.names = names
  
  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features


    return features, labels

In [0]:
def get_dataset(file, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file,
      batch_size = 50,
      label_name=LABEL_COLUMN,
      na_value ="?",
      num_epochs=1,
      ignore_errors=True,
      **kwargs)
  return dataset

In [0]:
LABEL_COLUMN = 'chd'
LABELS = [0, 1]

CATEGORIES = {'famhist': ['Present', 'Absent']}

In [266]:


SELECT_COLUMNS = ['sbp','tobacco','ldl','adiposity','famhist', 'typea','obesity','alcohol','age','chd']
raw_train_data = get_dataset(train_data, select_columns=SELECT_COLUMNS)
raw_test_data = get_dataset(test_data, select_columns=SELECT_COLUMNS)

TypeError: ignored

In [0]:
train_batch, label_batch = next(iter(raw_train_data))
test_batch, label_batch = next(iter(raw_test_data))

In [0]:
NUMERIC_FEATURES = ['sbp','tobacco','ldl','adiposity', 'typea','obesity','alcohol','age']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [0]:
train_batch, label_batch = next(iter(packed_train_data))
test_batch, label_batch = next(iter(packed_test_data))

In [0]:
desc = pd.read_csv(sys.argv[1])[NUMERIC_FEATURES].describe()

In [0]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])
def normalize_numeric_data(data, mean, std):
  return (data-mean)/std

In [0]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]

In [273]:
train_batch['numeric']

<tf.Tensor: id=160839, shape=(50, 8), dtype=float32, numpy=
array([[1.5800e+02, 3.6000e+00, 2.9700e+00, 3.0110e+01, 6.3000e+01,
        2.6640e+01, 1.0800e+02, 6.4000e+01],
       [1.5200e+02, 5.9900e+00, 7.9900e+00, 3.2480e+01, 4.5000e+01,
        2.6570e+01, 1.0032e+02, 4.8000e+01],
       [1.3600e+02, 3.4600e+00, 6.3800e+00, 3.2250e+01, 4.3000e+01,
        2.8730e+01, 3.1300e+00, 4.3000e+01],
       [1.5000e+02, 1.8000e-01, 4.1400e+00, 1.4400e+01, 5.3000e+01,
        2.3430e+01, 7.7100e+00, 4.4000e+01],
       [1.5600e+02, 0.0000e+00, 3.4700e+00, 2.1100e+01, 7.3000e+01,
        2.8400e+01, 0.0000e+00, 3.6000e+01],
       [1.6200e+02, 6.9400e+00, 4.5500e+00, 3.3360e+01, 5.2000e+01,
        2.7090e+01, 3.2060e+01, 4.3000e+01],
       [1.3400e+02, 6.4000e+00, 8.4900e+00, 3.7250e+01, 5.6000e+01,
        2.8940e+01, 1.0490e+01, 5.1000e+01],
       [1.2000e+02, 0.0000e+00, 5.0100e+00, 2.6130e+01, 6.4000e+01,
        2.6210e+01, 1.2240e+01, 3.3000e+01],
       [1.1800e+02, 1.2000e-01, 1.96

In [274]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(train_batch).numpy()

array([[ 0.9196036 ,  0.01833139, -0.8793434 ,  0.6086808 ,  1.05311   ,
         0.16786164,  3.648268  ,  1.4476614 ],
       [ 0.6305482 ,  0.5383526 ,  1.6714802 ,  0.9113547 , -0.7746761 ,
         0.15096642,  3.3395646 ,  0.36308125],
       [-0.1402663 , -0.01213008,  0.85338753,  0.8819814 , -0.9777634 ,
         0.672307  , -0.5670614 ,  0.02414994],
       [ 0.5341964 , -0.72579944, -0.28482878, -1.3976529 ,  0.03767331,
        -0.6069082 , -0.38296485,  0.0919362 ],
       [ 0.8232518 , -0.7649642 , -0.6252773 , -0.5419918 ,  2.0685468 ,
         0.59265774, -0.6928742 , -0.4503539 ],
       [ 1.1123072 ,  0.74505574, -0.0764944 ,  1.0237402 , -0.06387036,
         0.27647445,  0.595802  ,  0.02414994],
       [-0.2366181 ,  0.6275614 ,  1.9255463 ,  1.5205344 ,  0.34230432,
         0.72299314, -0.27122065,  0.56644005],
       [-0.9110808 , -0.7649642 ,  0.15724644,  0.10039238,  1.1546537 ,
         0.06407617, -0.20087807, -0.6537127 ],
       [-1.0074326 , -0.7388544 

In [0]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [0]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [0]:
model = tf.keras.Sequential([
  preprocessing_layer,
  layers.Dense(256, activation='relu',kernel_regularizer=regularizers.l2(0.005)),
  layers.Dropout(0.5),
   layers.Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.005)),
  layers.Dropout(0.5),
   layers.Dense(128, activation='selu',kernel_regularizer=regularizers.l2(0.005)),
  layers.Dropout(0.4),
  layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [282]:

print("--Train--")
model.fit(train_data, epochs=500)

test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy * 100))

--Train--
Epoch 1/500
9/9 [==============================] - 1s 89ms/step - loss: 2.2007 - accuracy: 0.6413
Epoch 2/500
9/9 [==============================] - 0s 8ms/step - loss: 2.0744 - accuracy: 0.6929
Epoch 3/500
9/9 [==============================] - 0s 8ms/step - loss: 1.9250 - accuracy: 0.6978
Epoch 4/500
9/9 [==============================] - 0s 8ms/step - loss: 1.8396 - accuracy: 0.7174
Epoch 5/500
9/9 [==============================] - 0s 8ms/step - loss: 1.8032 - accuracy: 0.7150
Epoch 6/500
9/9 [==============================] - 0s 8ms/step - loss: 1.6917 - accuracy: 0.7322
Epoch 7/500
9/9 [==============================] - 0s 7ms/step - loss: 1.5863 - accuracy: 0.7297
Epoch 8/500
9/9 [==============================] - 0s 8ms/step - loss: 1.5452 - accuracy: 0.7052
Epoch 9/500
9/9 [==============================] - 0s 8ms/step - loss: 1.4618 - accuracy: 0.7273
Epoch 10/500
9/9 [==============================] - 0s 9ms/step - loss: 1.4028 - accuracy: 0.7543
Epoch 11/500
9/9 [

In [283]:
# Show some results
predictions = model.predict(test_data)
for prediction, CHD in zip(predictions[:50], list(test_data)[0][1][:50]):
  print("Predicted Coronary Heart Disease: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("YES Coronary Heart Disease" if bool(CHD) else "NO Coronary Heart Disease"))

Predicted Coronary Heart Disease: 0.30%  | Actual outcome:  NO Coronary Heart Disease
Predicted Coronary Heart Disease: 2.13%  | Actual outcome:  NO Coronary Heart Disease
Predicted Coronary Heart Disease: 0.88%  | Actual outcome:  NO Coronary Heart Disease
Predicted Coronary Heart Disease: 0.60%  | Actual outcome:  NO Coronary Heart Disease
Predicted Coronary Heart Disease: 8.64%  | Actual outcome:  NO Coronary Heart Disease
Predicted Coronary Heart Disease: 2.75%  | Actual outcome:  YES Coronary Heart Disease
Predicted Coronary Heart Disease: 0.66%  | Actual outcome:  YES Coronary Heart Disease
Predicted Coronary Heart Disease: 1.30%  | Actual outcome:  YES Coronary Heart Disease
Predicted Coronary Heart Disease: 0.11%  | Actual outcome:  YES Coronary Heart Disease
Predicted Coronary Heart Disease: 0.59%  | Actual outcome:  YES Coronary Heart Disease
Predicted Coronary Heart Disease: 1.16%  | Actual outcome:  NO Coronary Heart Disease
Predicted Coronary Heart Disease: 96.87%  | Actua